In [1]:
from datasets import Dataset, DatasetDict, Audio, ClassLabel, Features
import numpy as np
import soundfile as sf
import os
import torch
import pandas as pd
from tqdm import tqdm
from transformers import ASTFeatureExtractor, ASTForAudioClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader
from datasets import Audio
import torchaudio

In [ ]:
os.environ["WANDB_API_KEY"] = 'your key'
os.environ["WANDB_PROJECT"] = "AST AIRI 228"
os.environ["WANDB_NOTES"] = "ASTSPOOF AIRI 228"
os.environ["WANDB_NAME"] = "astspoof-airi-228"

In [3]:
def get_ast_dataset(
    test_dir='/kaggle/input/safe-speak-2024-audio-spoof-detection-hackathon/wavs'
):
    valid_audio_files = []
    bad_names = []
    
    for file in tqdm(os.listdir(test_dir)):
        if file.endswith('.wav'):
            file_path = os.path.join(test_dir, file)
            waveform, sample_rate = torchaudio.load(file_path)
            duration = waveform.shape[1] / sample_rate
            if duration > 0.5:
                valid_audio_files.append(file_path)
            else:
                print(f"File: {file}, Duration: {duration:.2f} seconds")
                bad_names.append(file_path)
                
    test_dataset = Dataset.from_dict({
        'audio': valid_audio_files
    }, features=Features({
        'audio': Audio()
    }))
    
    return test_dataset, bad_names

In [4]:
#test_dataset = get_ast_dataset()

In [5]:
feature_extractor = ASTFeatureExtractor.from_pretrained('MIT/ast-finetuned-audioset-10-10-0.4593')
model = ASTForAudioClassification.from_pretrained('/kaggle/input/ast-airi-train/runs/ast_classifier/checkpoint-6834')
INPUT_NAME = feature_extractor.model_input_names[0]

def preprocess_audio(batch):
    wavs = [audio['array'] for audio in batch[INPUT_NAME]]
    inputs = feature_extractor(wavs, sampling_rate=feature_extractor.sampling_rate, return_tensors='pt')
    return {INPUT_NAME: inputs[INPUT_NAME]}

test_dataset, bad_names = get_ast_dataset()

test_dataset = test_dataset.cast_column('audio', Audio(sampling_rate=feature_extractor.sampling_rate))

test_dataset = test_dataset.rename_column('audio', INPUT_NAME)
test_dataset.set_transform(preprocess_audio, output_all_columns=False)

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir='./results',
        per_device_eval_batch_size=128
    )
)

test_predictions = trainer.predict(test_dataset)
logits = test_predictions.predictions
probabilities = torch.softmax(torch.tensor(logits), dim=-1).numpy()
class_0 = probabilities[:, 0]
class_1 = probabilities[:, 1]

preprocessor_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

  1%|          | 1301/144693 [00:21<37:57, 62.97it/s]

File: 69402.wav, Duration: 0.44 seconds


  5%|▍         | 7216/144693 [01:58<35:42, 64.16it/s]

File: 28216.wav, Duration: 0.35 seconds


...

File: 52907.wav, Duration: 0.47 seconds


 95%|█████████▌| 137980/144693 [38:05<01:36, 69.31it/s]

File: 28268.wav, Duration: 0.11 seconds


100%|██████████| 144693/144693 [39:59<00:00, 60.30it/s]


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: lightsource- (lightsource-unk). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20241127_123234-lp6xo82n
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run ./results
wandb: ⭐️ View project at https://wandb.ai/lightsource-unk/AST%20AIRI%20228
wandb: 🚀 View run at https://wandb.ai/lightsource-unk/AST%20AIRI%20228/runs/lp6xo82n


In [6]:
import os

In [ ]:
test_dir = '/kaggle/input/safe-speak-2024-audio-spoof-detection-hackathon/wavs/'
test_audio_files = [os.path.join(test_dir, file) for file in os.listdir(test_dir) if file.endswith('.wav')]
idxs = [int(test_audio_files[i].split('/')[-1][:-4]) for i in range(len(test_audio_files))]
idxs_bad = [int(bad_name.split('/')[-1][:-4]) for bad_name in bad_names]

df = pd.DataFrame({
    'ID': [x for x in idxs if x not in idxs_bad],
    'score': class_0
})

df_bad = pd.DataFrame({
    'ID': idxs_bad,
    'score': [0 for _ in range(len(idxs_bad))] # скорее всего короткие это спуф поэтому тут нули)
})

df = pd.concat([df, df_bad]).sort_values(by=['ID'])

df.to_csv('submission_ast_class1.csv', index=False)